 # 統計的仮説検定

In [1]:
# %
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
from matplotlib import pyplot as plt
import unittest
import doctest
from matplotlib import rcParams
import os
import warnings
warnings.simplefilter('ignore', FutureWarning)
path = os.path.dirname(os.path.abspath(__file__))

 ## 母平均に関する1標本の検定

 ### 統計的仮説検定の初歩

 #### 母平均に関する1標本のt検定

 #### 帰無仮説・対立仮設

 #### 有意差

 #### t検定の直感的な考え方

 #### 平均値の差が大きいだけでは有意差は得られない

 #### 検定統計量

 #### t値の復習

 #### ここまでのまとめその１

 #### 第一種の過誤・第二種の過誤

 #### 有意水準

 #### 棄却域・受容域

 #### p値

 #### ここまでのまとめその２

 #### t値とt分布の関係の復習

 #### 片側検定・両側検定

 #### 棄却域の計算方法

 #### p値の計算方法

 #### 数式を使ったまとめ

 #### 分析の準備

 #### t値の計算

 #### 棄却域の計算

 #### p値の計算

 #### シミュレーションによるp値の計算

 ### 平均値の差の検定

 #### 2群のデータに対するt検定

 #### 対応のあるt検定

 #### 分析の準備

 #### 対応のあるt検定

 #### 対応のないt検定（不等分散）

 #### 対応のないt検定（等分散）

 #### pハッキング

 ### 分割表の検定

 #### 分割表を用いるメリット

 #### 本章で扱う例題

 #### 期待度数との差を求める

 #### 分析の準備

 #### p値の計算

 #### 分割表の検定

 ### 検定の結果の解釈

 #### p値が0.05以下だったときの結果の書き方

 #### p値が0.05より大きかったときの結果の書き方

 #### 仮説検定における、よくある間違い

 #### p値が小さくても、差が大きいとは限らない

 #### p値が0.05より大きくても差がないとは言えない

 #### 検定の非対称性

 #### 有意水準は、検定をする前に決めておく

 #### 統計的仮説検定は必要か

 #### 仮説は正しいか